In [1]:
from pprint import pprint

In [2]:
from dataclasses import dataclass

In [3]:
from norvig_utils import *

In [4]:
@dataclass
class Scanner:
    readings: List
    pairs: List = None
    beacons: List = None

In [5]:
@dataclass
class Pair:
    a: Tuple
    b: Tuple
    vec: Tuple = None

In [6]:
@dataclass
class Overlap:
    s1: int
    s2: int
    rotidx: int
    cnt: int
    ofs: Tuple = None

In [7]:
parse_scanners = lambda filename: parse(filename, lambda lines: mapt(ints, lines.split('\n')[1:]), sep='\n\n')

In [8]:
def sub(a, b):
    return (a[0] - b[0], a[1] - b[1], a[2] - b[2])

In [9]:
def mnhdist(a, b):
    return sum(abs(a[i] - b[i]) for i in (0,1,2))

In [10]:
@cache
def make_rotated(x, y, z):
    return (
        (x, y, z), (x, z, -y), (x, -y, -z), (x, -z, y),
        (-x, -y, z), (-x, -z, -y), (-x, y, -z), (-x, z, y),
        (y, -x, z), (y, z, x), (y, x, -z), (y, -z, -x),
        (-y, x, z), (-y, -z, x), (-y, -x, -z), (-y, z, -x),
        (z, y, -x), (z, x, y), (z, -y, x), (z, -x, -y),
        (-z, y, x), (-z, -x, y), (-z, -y, -x), (-z, x, -y)
    )

In [11]:
# Check make_rotated on inputs from ex_rot.txt

views = parse_scanners('ex_rot.txt')

for points in transpose(views):
    base = points[0]
    rot = set(make_rotated(*points[0]))
    rest = set(points[1:])
    print(all(pt in rot for pt in rest))

----------------------------------------------------------------------------------------------------
ex_rot.txt ➜ 314 chars, 39 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7
----------------------------------------------------------------------------------------------------
parse() ➜ 5 entries:
----------------------------------------------------------------------------------------------------
(((-1, -1, 1), (-2, -2, 2), (-3, -3, 3), (-2, -3, 1), (5, 6, -4), (8,  ... -6, -4, -5), (0, 7, -8)))
----------------------------------------------------------------------------------------------------
True
True
True
True
True
True


In [12]:
# scanners = mapt(Scanner, parse_scanners('example.txt'))
scanners = mapt(Scanner, parse_scanners('input.txt'))

----------------------------------------------------------------------------------------------------
input.txt ➜ 10204 chars, 780 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
--- scanner 0 ---
-757,414,492
-593,762,-478
-608,779,-508
-761,323,468
-583,-536,626
539,660,453
----------------------------------------------------------------------------------------------------
parse() ➜ 28 entries:
----------------------------------------------------------------------------------------------------
(((-757, 414, 492), (-593, 762, -478), (-608, 779, -508), (-761, 323,  ... 457), (795, -714, -541)))
----------------------------------------------------------------------------------------------------


In [13]:
for s in scanners:
    s.pairs = [Pair(a, b, sub(a, b)) for a, b in combinations(s.readings, 2)]

In [14]:
overlaps = []

for i, base in enumerate(scanners):
    print('.', end='')
    for j, scanner in enumerate(scanners):
#         if i >= j:
        if i == j:
            continue
        rot_cnt = Counter()
        for pair in scanner.pairs:
            for k, d in enumerate(make_rotated(*pair.vec)):
                for b in base.pairs:
                    if b.vec == d:
                        rot_cnt[k] += 1
        if not rot_cnt:
            continue
        rot, cnt = rot_cnt.most_common(1)[0]
        if cnt < 11:
            continue

        overlaps.append(Overlap(i, j, rot, cnt))

............................

In [15]:
for ov in overlaps:
    print('.', end='')
    s1 = scanners[ov.s1]
    s2 = scanners[ov.s2]
    rotidx = ov.rotidx

    for p1 in s1.pairs:
        for p2 in s2.pairs:
            p2rot = make_rotated(*p2.vec)[rotidx]
            if p2rot == p1.vec:
                za = make_rotated(*p2.a)[rotidx]
                zb = make_rotated(*p2.b)[rotidx]
                ofs = sub(za, p1.a)
                assert(ofs == sub(zb, p1.b))
                if ov.ofs:
                    assert(ofs == ov.ofs)
                else:
                    ov.ofs = ofs

............................................................................................

In [16]:
overlapping_scanners = defaultdict(list)
for ov in overlaps:
    overlapping_scanners[ov.s1].append(ov.s2)
    
ovlookup = {(ov.s1, ov.s2): ov for ov in overlaps}

In [17]:
ovlookup

{(0, 1): Overlap(s1=0, s2=1, rotidx=21, cnt=37, ofs=(1291, 63, 13)),
 (0, 8): Overlap(s1=0, s2=8, rotidx=7, cnt=24, ofs=(91, 1379, -155)),
 (0, 9): Overlap(s1=0, s2=9, rotidx=5, cnt=27, ofs=(-44, 44, 1135)),
 (1, 0): Overlap(s1=1, s2=0, rotidx=15, cnt=37, ofs=(63, -13, 1291)),
 (1, 8): Overlap(s1=1, s2=8, rotidx=20, cnt=11, ofs=(-1316, -168, 1200)),
 (1, 11): Overlap(s1=1, s2=11, rotidx=8, cnt=31, ofs=(69, -156, -1192)),
 (1, 14): Overlap(s1=1, s2=14, rotidx=11, cnt=44, ofs=(21, 1227, 132)),
 (1, 20): Overlap(s1=1, s2=20, rotidx=23, cnt=37, ofs=(77, -1363, 5)),
 (1, 21): Overlap(s1=1, s2=21, rotidx=4, cnt=34, ofs=(-1317, -12, -15)),
 (2, 7): Overlap(s1=2, s2=7, rotidx=7, cnt=39, ofs=(-27, -1143, 16)),
 (2, 13): Overlap(s1=2, s2=13, rotidx=19, cnt=40, ofs=(-20, 1233, 57)),
 (2, 18): Overlap(s1=2, s2=18, rotidx=18, cnt=36, ofs=(31, 161, -1070)),
 (3, 4): Overlap(s1=3, s2=4, rotidx=5, cnt=38, ofs=(-47, -1192, 76)),
 (3, 14): Overlap(s1=3, s2=14, rotidx=8, cnt=28, ofs=(1269, 73, -20)),
 (3

In [18]:
overlapping_scanners

defaultdict(list,
            {0: [1, 8, 9],
             1: [0, 8, 11, 14, 20, 21],
             2: [7, 13, 18],
             3: [4, 14, 21],
             4: [3, 7, 15, 17, 18, 21],
             5: [22, 26],
             6: [10, 15, 16, 25],
             7: [2, 4, 11, 12, 14, 19],
             8: [0, 1, 21],
             9: [0, 14, 27],
             10: [6, 11, 15, 20],
             11: [1, 7, 10, 15],
             12: [7],
             13: [2, 22],
             14: [1, 3, 7, 9, 27],
             15: [4, 6, 10, 11, 21, 25],
             16: [6, 20, 21],
             17: [4, 19],
             18: [2, 4, 22],
             19: [7, 17],
             20: [1, 10, 16],
             21: [1, 3, 4, 8, 15, 16],
             22: [5, 13, 18],
             23: [24, 27],
             24: [23],
             25: [6, 15],
             26: [5],
             27: [9, 14, 23]})

In [19]:
def transform_point(p, rotidx, ofs):
    prot = make_rotated(*p)[rotidx]
    pmov = sub(prot, ofs)
    return pmov

In [20]:
def merge_scanners(dest, src):
    # uses globals: scanners, ovlookup
    ov = ovlookup[(dest, src)]
    
    s1 = scanners[ov.s1]
    s2 = scanners[ov.s2]
    rotidx = ov.rotidx
    ofs = ov.ofs

    s1.beacons |= {transform_point(p, rotidx, ofs) for p in s2.beacons}
    s2.beacons = set()
    
    s1.neighbors |= {transform_point(p, rotidx, ofs) for p in s2.neighbors}
    s2.neighbors = set()

In [21]:
for s in scanners:
    s.beacons = set(s.readings)
    s.neighbors = set(((0,0,0),))
    
print([len(scanners[i].beacons) for i in range(len(scanners))])
print([len(scanners[i].neighbors) for i in range(len(scanners))])

[26, 26, 26, 26, 25, 26, 25, 26, 26, 25, 26, 26, 26, 26, 25, 26, 26, 26, 26, 26, 27, 26, 26, 26, 26, 26, 26, 26]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
def dfs_merge(g):
    seen = set()
    
    def _dfs(n, p):
        if n in seen:
            return
        seen.add(n)
        for m in g[n]:
            _dfs(m, n)

        if p is not None:
            merge_scanners(p, n)
        
    _dfs(0, None)

dfs_merge(overlapping_scanners)
print([len(scanners[i].beacons) for i in range(len(scanners))])
print([len(scanners[i].neighbors) for i in range(len(scanners))])

[323, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [23]:
print('part 1 answer')
len(scanners[0].beacons)

part 1 answer


323

In [24]:
print('part 2 answer')

max([mnhdist(a, b) for a,b in combinations(scanners[0].neighbors, 2)])

part 2 answer


10685